# MNIST Experiments

This notebook is created to experiment with [MNIST](https://en.wikipedia.org/wiki/MNIST_database) dataset.

Plan:
- Dense network solution on CPU
- Dense network solution on GPU
- CNN+dense network solution on CPU
- CNN+dense network solution on GPU

Tools: Keras, PyTorch

In [1]:
import keras

Using TensorFlow backend.


In [85]:
from keras.datasets import mnist
#from keras.datasets import fashion_mnist
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [53]:
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [86]:
#get train & tesr dataset
(train_data, train_labels), (test_data, test_labels) = mnist.load_data()
#(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()

In [87]:
img_width = 28
img_height = 28
img_size = img_width * img_height # 784
train_img_number = train_data.shape[0]
test_img_number = test_data.shape[0]
classes_number = 10 # 10 digits 0-9
batch_size = 128
train_epochs = 10

In [88]:
# shape images
if K.image_data_format() == 'channels_first':
    train_data = train_data.reshape(train_img_number, 1, img_width, img_height)
    test_data = test_data.reshape(test_img_number, 1, img_width, img_height)
    input_shape = (1, img_width, img_height)
else: # channels_last
    train_data = train_data.reshape(train_img_number, img_width, img_height, 1)
    test_data = test_data.reshape(test_img_number, img_width, img_height, 1)
    input_shape = (img_width, img_height, 1)

In [89]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')
train_data /= 255
test_data /= 255

In [90]:
print('train shape', train_data.shape)

train shape (60000, 28, 28, 1)


In [91]:
train_labels = keras.utils.to_categorical(train_labels, classes_number)
test_labels = keras.utils.to_categorical(test_labels, classes_number)

In [127]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=input_shape)) # 32x28x28
model.add(Conv2D(64, kernel_size=(3,3), activation='relu')) # 64x28x28
model.add(MaxPooling2D(pool_size=(2,2))) # 64x14x14
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu')) # 128
model.add(Dropout(0.50))
model.add(Dense(classes_number, activation='softmax')) # 10

model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_36 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_28 (Flatten)         (None, 9216)              0         
_________________________________________________________________
dense_51 (Dense)             (None, 128)               1179776   
_________________________________________________________________
dropout_37 (Dropout)         (None, 128)             

In [121]:
model.compile(
    loss=keras.losses.categorical_crossentropy,
    optimizer=keras.optimizers.adadelta(), #keras.optimizers.adadelta(),
    metrics=['accuracy']
)

In [122]:
print(test_data.shape)
print(test_labels.shape)

(10000, 28, 28, 1)
(10000, 10)


In [123]:
train_epochs = 10

model.fit(
    train_data, train_labels, 
    batch_size=batch_size, 
    epochs=train_epochs, 
    verbose=1, 
    validation_data=(test_data, test_labels)
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 2s 30us/step - loss: 1.7500 - accuracy: 0.3865 - val_loss: 1.4751 - val_accuracy: 0.4943
Epoch 2/10
60000/60000 [==============================] - 2s 28us/step - loss: 1.3747 - accuracy: 0.5370 - val_loss: 1.2651 - val_accuracy: 0.5822
Epoch 3/10
60000/60000 [==============================] - 2s 28us/step - loss: 1.2352 - accuracy: 0.5888 - val_loss: 1.1862 - val_accuracy: 0.6143
Epoch 4/10
60000/60000 [==============================] - 2s 28us/step - loss: 1.1681 - accuracy: 0.6136 - val_loss: 1.1293 - val_accuracy: 0.6361
Epoch 5/10
60000/60000 [==============================] - 2s 28us/step - loss: 1.1276 - accuracy: 0.6267 - val_loss: 1.1006 - val_accuracy: 0.6477
Epoch 6/10
60000/60000 [==============================] - 2s 28us/step - loss: 1.1009 - accuracy: 0.6351 - val_loss: 1.0772 - val_accuracy: 0.6507
Epoch 7/10
60000/60000 [==============================] - 2s 28us/st

In [85]:
score = model.evaluate(test_data, test_labels, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.03707205359062646
Test accuracy: 0.9879999756813049
